MODELO DE MACHINE LEARNING MEDIANTE LIBRERIA SURPRISE PARA RECOMENDACION
DE RESTAURANTES BASADA EN LOS RATINGS OTORGADOS POR LOS USUARIOS

In [1]:
# Importamos librerías Pandas y Numpy para trabajar con dataframes y Glob y Os para
# el manejo de archivos
import pandas as pd
import numpy as np
import glob
import os

En primer lugar, unificaremos los datasets de reviews obtenidos de GCP en un solo archivo

In [2]:
# Generamos un listado de los archivos a unir recorriendo todos los archivos del path
# indicado y filtrando por el nombre pasado como parámetro con el uso de un comodín
joined_files = os.path.join("./datasets", "exportaciones_reviews*.csv")
joined_list = glob.glob(joined_files)
  
# Importamos todos los archivos y los concatenamos en un único dataframe
reviews = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

Cargaremos ahora el dataset con los datos de los comercios

In [ ]:
# Importamos el dataset
comercios = pd.read_csv("./datasets/metadata.csv")

In [5]:
# Eliminamos las columnas que no utilizaremos en el modelo
reviews = reviews.drop(["time","text","resp","state","resp_time","resp_text"], axis=1)

In [6]:
# Revisamos la composición del dataframe obtenido
comercios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329540 entries, 0 to 329539
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   name              329539 non-null  object
 1   address           328805 non-null  object
 2   gmap_id           329540 non-null  object
 3   description       22924 non-null   object
 4   latitude          329540 non-null  object
 5   longitude         329540 non-null  object
 6   avg_rating        329540 non-null  object
 7   num_of_reviews    329540 non-null  object
 8   price             49841 non-null   object
 9   hours             251993 non-null  object
 10  relative_results  269126 non-null  object
 11  url               329540 non-null  object
 12  cat_id            329540 non-null  object
 13  city_id           252539 non-null  object
dtypes: object(14)
memory usage: 35.2+ MB


In [7]:
# Eliminamos columnas adicionales
comercios = comercios.drop(["latitude","longitude","price","hours","relative_results","url"], axis=1)

Vamos a agregar ahora al dataframe de los comercios, datos adicionales como la categoría (por ejemplo, pizzeria) y el grupo (por ejemplo, alojamiento)

In [8]:
# Importamos la tabla de categorías
categorias = pd.read_csv("./datasets/categorias.csv")

In [9]:
# Importamos la tabla de grupos
grupos = pd.read_csv("./datasets/groups.csv")

In [10]:
# Hacemos un inner join de categorias con grupos para luego unirla a comercios
categorias = categorias.merge(grupos, on="group_id")

In [11]:
# Agregamos al dataframe de los comercios las categorias y grupos de cada uno de ellos
comercios = comercios.merge(categorias, on="cat_id")

A continuación, prepararemos el dataframe a introducir en el modelo del sistema
de recomendación

In [12]:
# Seleccionamos únicamente los sitios del grupo "comida", descartando "ocio" y "alojamiento"
comercios_comida = comercios[comercios["description_y"] == "Comida"]

In [13]:
# Agregamos al dataframe de reviews los datos de cada comercio 
reviews_completo = reviews.merge(comercios_comida, on="gmap_id")

In [18]:
# Importamos la tabla de ciudades para incorporlas luego a la tabla de comercios
# y poder filtrar en el modelo por la localidad seleccionada por el usuario
cities = pd.read_csv("./datasets/cities.csv")

In [21]:
# Preparamos el datrafame el modelo de ML. Al utilizar la librería Surprise, solo se
# pasan los datos relativos al usuario que hizo la review, el id del comercio visitado
# y el puntaje otorgado
reviews_modelo = reviews_completo[["user_id","gmap_id","rating"]]

In [ ]:
# Tomando en cuenta que usaremos el filtro de ciudad para seleccionar una recomendación,
# eliminamos los registros que no tienen ese dato
comercios_comida.dropna(subset=['city_id'], inplace=True)

In [23]:
# Cambiamos el tipo de dato para la columna "city_id" a efectos de poder 
# efectuar el join posterior
comercios_comida.astype({'city_id': 'int64'}).dtypes

name              object
address           object
gmap_id           object
description       object
avg_rating        object
num_of_reviews    object
cat_id            object
city_id            int64
description_x     object
group_id           int64
description_y     object
dtype: object

In [24]:
# Agregamos el nombre de la ciudad al dataframe de comercios
comercios_comida = comercios_comida.merge(cities, on="city_id")

Comenzamos ahora a trabajar directamente en el modelo de Machine Learning (ML)

In [25]:
# Importamos las dos librerías necesarias para nuestro modelo de ML de
# sistema de recomendación. 
# Surprise para efectuar las predicciones y Reader para indicarle como
# debe leer el dataframe que le ingestaremos. En este caso, le indicamos
# que el valor de los ratings va de 1 a 5
from surprise import Dataset
from surprise import Reader

reader = Reader(rating_scale=(1, 5))

# Ingestamos en el modelo de ML el dataset, con los 3 datos requeridos por Surprise
# para efectuar las predicciones
data = Dataset.load_from_df(reviews_modelo[['user_id', 'gmap_id', 'rating']], reader)

In [26]:
# Importamos la librería SVD y la de Validación cruzada para instanciar nuestro modelo
from surprise import SVD
from surprise.model_selection import cross_validate

# Instanciamos nuestro modelo y efectuamos una validación cruzada, obteniendo
# los diferentes resultados para cada instancia
svd = SVD(verbose=True, n_epochs=10)
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0423  1.0417  1.0413  1.0418  0.0004  
MAE (testset)     0.7864  0.7860  0.7859  0.7861  0.0002  
Fit time          14.63   17.85   16.87   16.45   1.34    
Test time         5.91    6.09    6.61    6.20    0.30    


{'test_rmse': array([1.04233623, 1.04166673, 1.04131331]),
 'test_mae': array([0.78638152, 0.78595958, 0.78593282]),
 'fit_time': (14.633838415145874, 17.84606695175171, 16.872409343719482),
 'test_time': (5.9072065353393555, 6.090474843978882, 6.614306926727295)}

In [27]:
# Entrenamos el modelo
trainset = data.build_full_trainset()
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


In [28]:
# Efectuamos una predicción de muestra, para un usuario y un negocio en particular
# En este caso, la predicción del rating para este usuario y negocio es de 4.6
svd.predict(uid=111293913029428063847, iid="0x89c6e40d71f55e27:0x9cc01b88f4ed07f8")

Prediction(uid=111293913029428063847, iid='0x89c6e40d71f55e27:0x9cc01b88f4ed07f8', r_ui=None, est=4.606893948087677, details={'was_impossible': False})

Por último, exportaremos el modelo entrenado y el dataset utilizado para que
se carguen en otra instancia y sean puestos en producción

In [32]:
# Importamos la librería Pickle que nos permite guardar el modelo entrenado en un 
# archivo de extensión pkl
import pickle
pickle.dump(svd,open("model.pkl","wb"))

In [33]:
# Exportamos el dataset final de los comercios del grupo "comida" para ser
# utilizado en otra instancia
comercios_comida.to_csv("comercios_comida.csv")